# Import Packages

In [85]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import tensorflow as tf
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deolu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deolu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Note

For the Essentials Badge, I'm going to do two different approaches. Both will involve deep learning, however one will try to limit the alteration to the original input data as much as possible in order to do a straight comparison. The second method will be a more involved approach at pre-proccessing the data before feeding it into the model

In [2]:
file = open('jeopardy.json')
rawData = json.load(file)

corpus = [entry['question'].lower() for entry in rawData if entry['value']!=None]
corpus[0:2]


["'for the last 8 years of his life, galileo was under house arrest for espousing this man's theory'",
 "'no. 2: 1912 olympian; football star at carlisle indian school; 6 mlb seasons with the reds, giants & braves'"]

In [46]:
worth = [1 if int(item['value'].strip('$').replace(',','')) >= 500 else 0 for item in rawData if item['value']!=None]
# worth = [int(num['value'].strip('$')) for num in worth]

worth = np.array(worth)
worth.shape


(213296,)

In [47]:
train_x, test_x, train_y, test_y = train_test_split(corpus, worth, random_state=1)
# print("Training Size: " + str(len(train_x)))
np.array(train_x).shape

(159972,)

In [41]:
vectorizer = HashingVectorizer(n_features=2**10)
# vectorizer = CountVectorizer(stop_words='english')
trainVectorized = vectorizer.fit_transform(train_x)
testVectorized = vectorizer.transform(test_x)
trainVectorized = np.array(trainVectorized.toarray())
testVectorized = np.array(testVectorized.toarray())


In [51]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [52]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [53]:
model.fit(trainVectorized, train_y, epochs=20)

Epoch 1/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.6748 - accuracy: 0.5771
Epoch 2/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.6650 - accuracy: 0.5956
Epoch 3/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.6378 - accuracy: 0.6355
Epoch 4/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.5867 - accuracy: 0.6921
Epoch 5/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.5237 - accuracy: 0.7439
Epoch 6/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.4642 - accuracy: 0.7848
Epoch 7/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.4114 - accuracy: 0.8160
Epoch 8/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.3661 - accuracy: 0.8423
Epoch 9/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.3269 - accuracy: 0.8640
Epoch 10/20
5000/5000 [==============================] - 7s 1ms/step - loss: 0.292

In [54]:
test_loss, test_acc = model.evaluate(testVectorized,  test_y, verbose=2)

print('\nTest accuracy:', test_acc)

1667/1667 - 2s - loss: 2.2073 - accuracy: 0.5321 - 2s/epoch - 1ms/step

Test accuracy: 0.5321431159973145


# Let's Try a Different method

For this method, I used a different method where I increased the number of possible values from 2 ("low" or "high") to 10 (actual values). I simply used 10 different classes for the inputs. You can see how I assigned them in the module below. I then used a different way of measuring error as well. 

Furthermore, I used a different Vecotrizer. I used a hash vectorizer. This had more to do with limiting the size of the matrix than anything else

Overall, this gave me a much better result. I belive this has more to do with the extension of the possible values than the diffrent error metric or the hash vectorizer. Given that these values come from jeopardy, there should be some rationale (common theme) between the questions that can be identified.

In [111]:
worthExtra = [int(item['value'].strip('$').replace(',','')) for item in rawData if item['value']!=None]
worthConverted = []
i = 0
j = 0
for value in worthExtra:
    if(value<=100):
        worthConverted.append([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    elif (value>100 & value <= 200):
        worthConverted.append([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
    elif (value==300):
        worthConverted.append([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    elif (value==400):
        worthConverted.append([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
    elif (value==500):
        worthConverted.append([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
    elif (value==600):
        worthConverted.append([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
    elif (value==700):
        worthConverted.append([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
    elif (value==800):
        worthConverted.append([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
    elif (value==900):
        worthConverted.append([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
    elif (value>=1000):
        worthConverted.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
    else:
        j = j + 1
    i = i + 1
    # if (i == len(worthExtra)):
        # print(j)
worthConverted = np.array(worthConverted)
# len(worthConverted)
worthConverted.shape
# len(worthExtra)
# min(worthExtra)

(213296, 10)

In [97]:
worth.shape

(213296,)

In [119]:
train_x2, test_x2, train_y2, test_y2 = train_test_split(corpus, worthConverted, random_state=1)

In [120]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [126]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [127]:
model2.fit(trainVectorized, train_y2, epochs=20)

Epoch 1/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0081 - accuracy: 0.9577
Epoch 2/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0081 - accuracy: 0.9577
Epoch 3/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0080 - accuracy: 0.9577
Epoch 4/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0079 - accuracy: 0.9577
Epoch 5/20
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0078 - accuracy: 0.9577
Epoch 6/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0076 - accuracy: 0.9578
Epoch 7/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0074 - accuracy: 0.9581
Epoch 8/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0072 - accuracy: 0.9587
Epoch 9/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.0069 - accuracy: 0.9595
Epoch 10/20
5000/5000 [==============================] - 8s 2ms/step - loss: 0.006

In [128]:
test_loss2, test_acc2 = model2.evaluate(testVectorized,  test_y2, verbose=2)

print('\nTest accuracy:', test_acc2)

1667/1667 - 2s - loss: 0.0106 - accuracy: 0.9454 - 2s/epoch - 1ms/step

Test accuracy: 0.9454467296600342
